<a href="https://colab.research.google.com/github/Ilvecho/Project_6/blob/main/Project_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TTS

from TTS.api import TTS
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import torch
from google.colab import files,drive

drive.mount('/content/gdrive')

# Pre-trained TTS model

List of all available TTS pre-trained models in **English**

 - 'tts_models/en/ek1/tacotron2',
 - 'tts_models/en/ljspeech/tacotron2-DDC',
 - 'tts_models/en/ljspeech/tacotron2-DDC_ph',
 - 'tts_models/en/ljspeech/glow-tts',
 - 'tts_models/en/ljspeech/speedy-speech',
 - 'tts_models/en/ljspeech/tacotron2-DCA',
 - 'tts_models/en/ljspeech/vits',
 - 'tts_models/en/ljspeech/vits--neon',
 - 'tts_models/en/ljspeech/fast_pitch',
 - 'tts_models/en/ljspeech/overflow',
 - 'tts_models/en/ljspeech/neural_hmm',
 - 'tts_models/en/vctk/vits',
 - 'tts_models/en/vctk/fast_pitch',
 - 'tts_models/en/sam/tacotron-DDC',
 - 'tts_models/en/blizzard2013/capacitron-t2-c50',
 - 'tts_models/en/blizzard2013/capacitron-t2-c150_v2',
 - 'tts_models/en/multi-dataset/tortoise-v2',

In [ ]:
from TTS.api import TTS

device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS('tts_models/multilingual/multi-dataset/your_tts').to(device)

In [12]:
speaker = "/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/"

text = {}
audio_dir = {}

for file_name in os.listdir(speaker):

  sentence, ext = file_name.split('.')

  if ext == 'TXT':
    with open(speaker + file_name, 'r') as file:
      line = file.readline()
      parts = line.split(' ')
      # The sentences end with a new line simbol, we need to drop it
      text[sentence] = ' '.join(parts[2:]).strip('\n')

  elif ext == 'WAV':
    audio_dir[sentence] = speaker + file_name

print(text)
print(audio_dir)

{'SA1': 'She had your dark suit in greasy wash water all year.', 'SA2': "Don't ask me to carry an oily rag like that.", 'SI1049': 'They understood and teased me a bit about it.', 'SI1679': 'Deep enough, he decided.', 'SI2309': "Well, wait until you finish what you've got, Davy.", 'SX149': 'While waiting for Chipper she crisscrossed the square many times.', 'SX239': 'Puree some fruit before preparing the skewers.', 'SX329': 'Chip postponed alimony payments until the latest possible date.', 'SX419': 'The football team coach has a watch thin as a dime.', 'SX59': "The singer's finger had a splinter."}
{'SA1': '/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/SA1.WAV', 'SA2': '/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/SA2.WAV', 'SI1049': '/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/SI1049.WAV', 'SI1679': '/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/SI1679.WAV', 'SI2309': '/content/gdrive/MyDrive/P6_files/TIMIT/TRAIN/DR7/MSAH1/SI2309.WAV', 'SX149': '/

In [13]:
with open('/content/gdrive/MyDrive/P6_files/text.pkl', 'wb') as handle:
  pickle.dump(text, handle)

with open('/content/gdrive/MyDrive/P6_files/audio_dir.pkl', 'wb') as handle:
  pickle.dump(audio_dir, handle)

In [15]:
for i, key in enumerate(text.keys()):
  # generate speech by cloning a voice using default settings
  tts.tts_to_file(text=text[key],
                  speaker_wav=audio_dir[key],
                  language='en',
                  file_path=f"/content/gdrive/MyDrive/P6_files/test_same_text/output_{i}.wav") # speed=1.15

 > Text splitted to sentences.
['She had your dark suit in greasy wash water all year.']
 > Processing time: 2.1620981693267822
 > Real-time factor: 0.6720852251559782
 > Text splitted to sentences.
["Don't ask me to carry an oily rag like that."]
 > Processing time: 2.944546699523926
 > Real-time factor: 1.1868386535767537
 > Text splitted to sentences.
['They understood and teased me a bit about it.']
 > Processing time: 3.0338845252990723
 > Real-time factor: 1.148763546118543
 > Text splitted to sentences.
['Deep enough, he decided.']
 > Processing time: 2.158935546875
 > Real-time factor: 1.0217394921320397
 > Text splitted to sentences.
["Well, wait until you finish what you've got, Davy."]
 > Processing time: 2.0889780521392822
 > Real-time factor: 0.7210832075040671
 > Text splitted to sentences.
['While waiting for Chipper she crisscrossed the square many times.']
 > Processing time: 2.4473066329956055
 > Real-time factor: 0.7046664650145711
 > Text splitted to sentences.
['Pu

In [23]:
from IPython.display import Audio


path = '/content/gdrive/MyDrive/P6_files/test_same_text/'

for file_name in os.listdir(path):
  print(path + file_name)
  Audio(path + file_name, rate=22050)
  break

/content/gdrive/MyDrive/P6_files/test_same_text/output_0.wav
